### Esercitazione 6/11/2024

In [3]:
type poitn_2d_3d =
    | P2dim of float*float
    | P3dim of float*float*float ;;
    
type 'a box =
    | Empty
    | Full of 'a
    

type 'a colored_tree = 
    | Black of 'a
    | Red of 'a*('a colored_tree)*('a colored_tree) 
    | Blue of 'a*('a colored_tree)*('a colored_tree) ;;
    


type poitn_2d_3d = P2dim of float * float | P3dim of float * float * float


type 'a box = Empty | Full of 'a


type 'a colored_tree =
    Black of 'a
  | Red of 'a * 'a colored_tree * 'a colored_tree
  | Blue of 'a * 'a colored_tree * 'a colored_tree


In [4]:
(* Assumendo le definizioni sopra riportate, determinare il tipo delle seguenti espressioni *)

(Some 10)::(None)::[] ;; (* int option list *)
(Full 10)::(Empty)::[] ;; (* int box list *)


let rec f ct = match ct with
    | Black n -> 0
    | Red (n, ct1, ct2) -> n + (f ct1) + (f ct2)    
    | Blue (n, ct1, ct2) -> (f ct1) + (f ct2) ;; (* fun : colored_tree -> int*)
    
let g p1 p2 = match p1, p2 with
    | P2dim (x1, y1), P2dim (x2, y2) -> Some (P2dim ((x1 +. x2)/.2., (y2 +. y2)/.2.))
    | P3dim (x1, y1, z1), P3dim (x2, y2, z2) -> Some (P3dim ((x1 +. x2)/.2., (y2 +. y2)/.2., (z1 +. z2)/.2.))
    | _,_ -> None ;; 
    
(* function: point2d_3d -> point2d_3d -> option*)

- : int option list = [Some 10; None]


- : int box list = [Full 10; Empty]


val f : int colored_tree -> int = <fun>


val g : poitn_2d_3d -> poitn_2d_3d -> poitn_2d_3d option = <fun>


#### Cosa calcolano le funzioni f e g riportate sopra?

f calcola la somma dei nodi se Rossi;
g calcola il punto medio a seconda del tipo dei punti passati.

#### Esercizio 
Considerando il tipo ’a colored_tree che rappresenta alberi con nodi colorati definito
nell’esercizio 1, scrivere una funzione raccogli che, dato un albero di questo tipo, restituisce una tripla di
liste (l1,l2,l3) di tipo ’a list * ’a list * ’a list dove l1 contiene tutti i valori contenuti nei nodi
Black, l2 tutti i valori nei nodi Red ed l3 tutti i valori nei nodi Blue. Ad esempio, dato il seguente albero:


let tree = Red ("a",
Blue ("b",
Black "c",
Red ( "d", Black "e", Black "f")),
Red ( "g",
Blue ( "h", Black "i", Black "j"),
Black "k")) ;;


l’applicazione raccogli tree deve dare come risultato:
(["c"; "e"; "f"; "i"; "j"; "k"], ["a"; "d"; "g"], ["b"; "h"])
L’ordine degli elementi nelle liste può essere diverso rispetto a questo esempio.

In [5]:
let rec raccogli tree = 
    match tree with 
    | Black n -> ([n], [], []) 
    | Red (n, sx, dx) -> let (l1, l2, l3) = raccogli sx in 
                         let (l1', l2', l3') = raccogli dx in
                         (l1@l1', n::l2@l2', l3@l3')
    | Blue (n, sx, dx) -> let (l1, l2, l3) = raccogli sx in 
                          let (l1', l2', l3') = raccogli dx in
                          (l1@l1', l2@l2', n::l3@l3') ;;
    
let tree = Red ("a", Blue ("b", Black "c", Red ( "d", Black "e", Black "f")), Red ( "g", Blue ( "h", Black "i", Black "j"), Black "k")) ;;
raccogli tree

val raccogli : 'a colored_tree -> 'a list * 'a list * 'a list = <fun>


val tree : string colored_tree =
  Red ("a", Blue ("b", Black "c", Red ("d", Black "e", Black "f")),
   Red ("g", Blue ("h", Black "i", Black "j"), Black "k"))


- : string list * string list * string list =
(["c"; "e"; "f"; "i"; "j"; "k"], ["a"; "d"; "g"], ["b"; "h"])


#### Esercizio

In [6]:
type btree =
| Void
| Node of int * btree * btree

type btree = Void | Node of int * btree * btree


si definisca, usando i costrutti di programmazione funzionale di OCaml, una funzione flat con tipo
flat : btree -> int list list
tale che flat bt restituisca una lista contenente le liste di valori presenti ad ogni livello di profondità
dell’albero.
Ad esempio dato il seguente albero binario (a destra in una rappresentazione visuale):


let bt =
Node (3,
Node (5,
Node(1,Void,Void),
Void
),
Node (-4,
Node(6,Void,Void),
Node(8,Void,Void)
)
)


abbiamo che flat bt restituisce [[3],[5,-4],[1,6,8]].

In [36]:
(* flatta un binary tree... Ops*)
let flat bt =
  let rec helper bt acc = 
    match bt with 
    | Void -> acc
    | Node(data, sx, dx) -> helper sx (data :: helper dx acc)
  in
  helper bt [] ;;
        
let bt = Node (3, Node (5, Node(1,Void,Void), Void ), Node (-4, Node(6,Void,Void), Node(8,Void,Void) ) ) ;;
flat bt

val flat : btree -> int list = <fun>


val bt : btree =
  Node (3, Node (5, Node (1, Void, Void), Void),
   Node (-4, Node (6, Void, Void), Node (8, Void, Void)))


- : int list = [1; 5; 3; 6; -4; 8]


In [43]:
(* Invece vogliamo i valori dei nodi ad ogni livello. Soluzione proposta dal Prof. Milazzo *) 

let rec flat bt =
    let rec merge lis1 lis2 =
        match (lis1, lis2) with
            | ([], _) -> lis2
            | (_, []) -> lis1
            | (x::lis1', y::lis2') -> (x @ y) :: (merge lis1' lis2')  
    in
    match bt with
        | Void -> [] 
        | Node (n, bt1, bt2) -> 
            let lis1 = flat bt1 in
            let lis2 = flat bt2 in
            [[n]] @ (merge lis1 lis2)

let bt = Node (3, Node (5, Node(1,Void,Void), Void ), Node (-4, Node(6,Void,Void), Node(8,Void,Void) ) ) ;;
flat bt 

val flat : btree -> int list list = <fun>


val bt : btree =
  Node (3, Node (5, Node (1, Void, Void), Void),
   Node (-4, Node (6, Void, Void), Node (8, Void, Void)))


- : int list list = [[3]; [5; -4]; [1; 6; 8]]
